In [3]:
import requests
import csv
import ast
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from decimal import Decimal
import pickle

In [4]:
# Extracting data from the API, you might need your own APIID and APPLICATIONKEY , API link - https://developer.edamam.com/edamam-docs-recipe-api

APPID = '1283c1ab'
APPLICATIONKEY = '6ed575d941bbbea2d4e50379f436c326'
URL = 'https://api.edamam.com/api/recipes/v2'

def create_data(mealtype, read_mode):

  response = requests.get(f"{URL}?app_key={APPLICATIONKEY}&app_id={APPID}&mealType={mealtype}&type=any")

  if response.status_code == 200:
      data = response.json()
      suggested_recipes1 = []
      for i in range(20):
          suggested_recipes1.append(data['hits'][i]['recipe'])

      for _ in range(40):
          if( data['_links']['next']['href']):
              response = requests.get(data['_links']['next']['href'])
              if response.status_code == 200:
                  data = response.json()
                  for i in range(20):
                      suggested_recipes1.append(data['hits'][i]['recipe'])
              else:
                  print(f"error {response.status_code}")
          else:
              break

      print('result returned')

      print(len(suggested_recipes1))
      labels = []
      protein = []
      fat = []
      energy = []
      carbs = []
      fibres = []
      image_url = []
      ingridients = []
      mealtype = []
      url = []
      cuisinetype = []
      diet = []


      # Put the data in proper format for creation of csv
      for i in range(len(suggested_recipes1)):
          try:
              labels.append(suggested_recipes1[i]['label'])
              protein.append(suggested_recipes1[i]['totalNutrients']['PROCNT']['quantity'])
              fat.append(suggested_recipes1[i]['totalNutrients']['FAT']['quantity'])
              energy.append(suggested_recipes1[i]['totalNutrients']['ENERC_KCAL']['quantity'])
              carbs.append(suggested_recipes1[i]['totalNutrients']['CHOCDF']['quantity'])
              fibres.append(suggested_recipes1[i]['totalNutrients']['FIBTG']['quantity'])
              image_url.append(suggested_recipes1[i]['image'])
              ingridients.append(suggested_recipes1[i]['ingredientLines'])
              mealtype.append(suggested_recipes1[i]['mealType'])
              url.append(suggested_recipes1[i]['url'])
              cuisinetype.append(suggested_recipes1[i]['cuisineType'])
              diet.append(suggested_recipes1[i]['dietLabels'])
          except:
              print("This row is not appended")

      print('appended in list')
      db_columns = zip(labels, protein, fat, energy, carbs, fibres, image_url, ingridients, mealtype, url, cuisinetype, diet)

      # Create a csv file from the extracted data
      csv_file_path = 'dataset.csv'
      with open(csv_file_path, read_mode, newline='') as csv_file:
          writer = csv.writer(csv_file)

          if read_mode == 'w':
              writer.writerow(['Name', 'Protein', 'Fats', 'Energy', 'Carbs', 'Fibres', 'image_url', 'ingridients', 'mealtype', 'url', 'cuisinetype', 'diet'])

          for row in db_columns:
              try:
                  writer.writerow(row)
              except:
                  print('encoding error')
                  continue
      print('csv created')

In [5]:
# create the dataset - run only 1st time

create_data('Breakfast', 'w')
create_data('Snack', 'a')
create_data('Teatime', 'a')
create_data('Lunch', 'a')
create_data('Dinner', 'a')

result returned
820
appended in list
csv created
result returned
820
appended in list
csv created
result returned
820
This row is not appended
appended in list
csv created
result returned
820
appended in list
csv created
result returned
820
appended in list
csv created


preprocessing

In [52]:
df = pd.read_csv('dataset.csv')

In [53]:
df.shape

(4099, 12)

In [54]:
df.head()

,Name,Protein,Fats,Energy,Carbs,Fibres,image_url,ingridients,mealtype,url,cuisinetype,diet
0,Easy Mango-Blackberry Smoothie Recipe,11.235393,9.032777,274.596327,41.249251,8.635761,https://edamam-product-images.s3.amazonaws.com...,"['1 large ripe Ataulfo mango, peeled and rough...",['breakfast'],http://www.seriouseats.com/recipes/2014/01/eas...,['american'],"['Balanced', 'High-Fiber', 'Low-Sodium']"
1,Spelt Everything Crackers,38.106000,6.342300,882.180000,183.222000,27.927000,https://edamam-product-images.s3.amazonaws.com...,"['1/4 teaspoon salt', '1 1/2 cups white spelt ...",['breakfast'],http://smittenkitchen.com/2008/12/spelt-everyt...,['american'],"['Low-Fat', 'Low-Sodium']"
2,Seville Orange Marmalade,12.791305,1.632933,5905.772657,1521.348809,32.658651,https://edamam-product-images.s3.amazonaws.com...,"['3 pounds Seville oranges', '2 quarts water',...",['breakfast'],http://www.lottieanddoof.com/2013/03/seville-o...,['american'],"['Low-Fat', 'Low-Sodium']"
3,Andy Ricker's Naam Cheuam Naam Taan Piip (Palm...,0.085049,0.000000,269.320470,69.527205,0.000000,https://edamam-product-images.s3.amazonaws.com...,"['2 1/2 ounces palm sugar, coarsely chopped', ...",['breakfast'],https://www.seriouseats.com/recipes/2013/11/an...,['american'],"['Low-Fat', 'Low-Sodium']"
4,Sweet Rice Flour Porridge from 'The Kimchi Coo...,1.175125,0.280450,72.285000,15.819750,0.474000,https://edamam-product-images.s3.amazonaws.com...,"['1 cup water, divided', '2 tablespoons sweet ...","['breakfast', 'lunch/dinner']",http://www.seriouseats.com/recipes/2013/01/swe...,"['american', 'korean']","['Low-Fat', 'Low-Sodium']"


In [55]:
df.describe()

,Protein,Fats,Energy,Carbs,Fibres
count,4099.000000,4099.000000,4099.000000,4099.000000,4099.000000
mean,34.662090,69.371379,1613.125322,211.380404,12.958831
std,73.691642,138.847032,1888.331202,273.347194,17.927681
min,0.000000,0.000000,0.060000,0.000000,0.000000
25%,2.613883,0.850243,332.539906,22.252701,0.721049
50%,11.932640,11.363321,998.310000,116.414840,7.239820
75%,45.869812,92.586268,2205.916540,299.507782,17.997450
max,1148.378556,2614.627513,24588.334131,3363.657600,264.704000


In [56]:
df.dtypes # Note the data type of nutrient columns that is from 2 to 6 are float

Name            object
Protein        float64
Fats           float64
Energy         float64
Carbs          float64
Fibres         float64
image_url       object
ingridients     object
mealtype        object
url             object
cuisinetype     object
diet            object
dtype: object

Removing Duplicates

In [57]:
df = df.drop_duplicates(subset=['Name']) # drop any duplicate recipes which may be extracted from API

In [58]:
df.isnull().sum() # check for null values in columns and if so drop those rows

Name           0
Protein        0
Fats           0
Energy         0
Carbs          0
Fibres         0
image_url      0
ingridients    0
mealtype       0
url            0
cuisinetype    0
diet           0
dtype: int64

In [59]:
df = df.dropna()

In [60]:
df.isnull().sum()

Name           0
Protein        0
Fats           0
Energy         0
Carbs          0
Fibres         0
image_url      0
ingridients    0
mealtype       0
url            0
cuisinetype    0
diet           0
dtype: int64

In [61]:
columns = ['Protein', 'Fats', 'Energy', 'Carbs', 'Fibres']

In [62]:
#  convert the nutrient columns to integer
for col in columns:
  df[col] = pd.to_numeric(df[col], errors='coerce').astype(int).round(2)

export csv here to store in local database

Continue with furthe processing for applying nearest neighbours algo

In [63]:
df['mealtype'].unique() # All the Categories of mealtype

array(["['breakfast']", "['breakfast', 'lunch/dinner']",
       "['breakfast', 'brunch']", "['breakfast', 'snack']", "['snack']",
       "['lunch/dinner', 'snack']", "['teatime', 'snack']",
       "['snack', 'lunch/dinner']", "['brunch', 'snack']",
       "['teatime', 'lunch/dinner', 'snack']", "['teatime']",
       "['teatime', 'lunch/dinner']", "['breakfast', 'teatime']",
       "['lunch/dinner']", "['lunch/dinner', 'brunch']"], dtype=object)

In [64]:
df['cuisinetype'].unique() # All the Categories of cuisinetype

array(["['american']", "['american', 'korean']", "['mediterranean']",
       "['indian']", "['british']", "['french']", "['mexican']",
       "['eastern europe']", "['south east asian', 'asian', 'korean']",
       "['mediterranean', 'japanese']", "['kosher']", "['italian']",
       "['south american']", "['asian', 'korean']",
       "['american', 'british']", "['american', 'french']", "['nordic']",
       "['south east asian']", "['chinese']",
       "['south american', 'korean']", "['mediterranean', 'greek']",
       "['south east asian', 'chinese']", "['middle eastern']",
       "['asian']", "['central europe']", "['american', 'mexican']",
       "['japanese']", "['british', 'french']",
       "['american', 'south east asian']",
       "['south east asian', 'korean']", "['caribbean']",
       "['italian', 'middle eastern']", "['american', 'middle eastern']",
       "['american', 'nordic']", "['chinese', 'american']",
       "['american', 'italian']", "['south american', 'mexican']",


convert string to array for columns mealtype, diet, cuisinetype eg -> "['a', 'b']" to ['a','b']

In [65]:
def string_to_list(string):
    return ast.literal_eval(string)

columns = ['mealtype', 'cuisinetype', 'diet', 'ingridients']

for col in columns:
  df[col] = df[col].apply(string_to_list)

encoding columns

In [66]:
#  label encoding was done and a separate column was created for each category
columns = ['mealtype', 'cuisinetype', 'diet']

for col in columns:
  df[col] = df[col].apply(lambda x: ','.join(x))
  dummy = df[col].str.get_dummies(sep=',')
  df = pd.concat([df, dummy], axis=1)

In [67]:
df.dtypes

Name                object
Protein              int64
Fats                 int64
Energy               int64
Carbs                int64
Fibres               int64
image_url           object
ingridients         object
mealtype            object
url                 object
cuisinetype         object
diet                object
breakfast            int64
brunch               int64
lunch/dinner         int64
snack                int64
teatime              int64
american             int64
asian                int64
british              int64
caribbean            int64
central europe       int64
chinese              int64
eastern europe       int64
french               int64
greek                int64
indian               int64
italian              int64
japanese             int64
korean               int64
kosher               int64
mediterranean        int64
mexican              int64
middle eastern       int64
nordic               int64
south american       int64
south east asian     int64
w

In [68]:
df.to_csv('cleaned.csv') # preprocessing completed, copy the clean data to a new csv, this is later used in backend for creatiing the mongoDB database

In [50]:
# df = df.rename(columns={df.columns[0]: 'id'})

---------------------------------------------------------------- pre processing ends ----------------------------------------------

Training the Model

In [74]:
df = pd.read_csv('cleaned.csv')
df.head(2)
# here you can save the cleaned.csv if you want, manually rename the unnamed column to 'id'

,Unnamed: 0,Name,Protein,Fats,Energy,Carbs,Fibres,image_url,ingridients,mealtype,...,nordic,south american,south east asian,world,Balanced,High-Fiber,High-Protein,Low-Carb,Low-Fat,Low-Sodium
0,0,Easy Mango-Blackberry Smoothie Recipe,11,9,274,41,8,https://edamam-product-images.s3.amazonaws.com...,"['1 large ripe Ataulfo mango, peeled and rough...",breakfast,...,0,0,0,0,1,1,0,0,0,1
1,1,Spelt Everything Crackers,38,6,882,183,27,https://edamam-product-images.s3.amazonaws.com...,"['1/4 teaspoon salt', '1 1/2 cups white spelt ...",breakfast,...,0,0,0,0,0,0,0,0,1,1


In [76]:
df = df.drop(columns=df.columns[0])
df.head()

,Name,Protein,Fats,Energy,Carbs,Fibres,image_url,ingridients,mealtype,url,...,nordic,south american,south east asian,world,Balanced,High-Fiber,High-Protein,Low-Carb,Low-Fat,Low-Sodium
0,Easy Mango-Blackberry Smoothie Recipe,11,9,274,41,8,https://edamam-product-images.s3.amazonaws.com...,"['1 large ripe Ataulfo mango, peeled and rough...",breakfast,http://www.seriouseats.com/recipes/2014/01/eas...,...,0,0,0,0,1,1,0,0,0,1
1,Spelt Everything Crackers,38,6,882,183,27,https://edamam-product-images.s3.amazonaws.com...,"['1/4 teaspoon salt', '1 1/2 cups white spelt ...",breakfast,http://smittenkitchen.com/2008/12/spelt-everyt...,...,0,0,0,0,0,0,0,0,1,1
2,Seville Orange Marmalade,12,1,5905,1521,32,https://edamam-product-images.s3.amazonaws.com...,"['3 pounds Seville oranges', '2 quarts water',...",breakfast,http://www.lottieanddoof.com/2013/03/seville-o...,...,0,0,0,0,0,0,0,0,1,1
3,Andy Ricker's Naam Cheuam Naam Taan Piip (Palm...,0,0,269,69,0,https://edamam-product-images.s3.amazonaws.com...,"['2 1/2 ounces palm sugar, coarsely chopped', ...",breakfast,https://www.seriouseats.com/recipes/2013/11/an...,...,0,0,0,0,0,0,0,0,1,1
4,Sweet Rice Flour Porridge from 'The Kimchi Coo...,1,0,72,15,0,https://edamam-product-images.s3.amazonaws.com...,"['1 cup water, divided', '2 tablespoons sweet ...","breakfast,lunch/dinner",http://www.seriouseats.com/recipes/2013/01/swe...,...,0,0,0,0,0,0,0,0,1,1


Apply nearest neighbours (only nutrients)

In [77]:
neigh = NearestNeighbors(n_neighbors=5)

# use only nutrients to calculate nearest neighbours
neigh.fit(df.iloc[:, list(range(1, 6))].to_numpy())

NearestNeighbors()

saving the 1st model

In [78]:
with open('recipemodel', 'wb') as file:
    pickle.dump(neigh, file)

In [79]:
model = pickle.load(open('recipemodel','rb'))

Pred

In [80]:
def create_input(row):
    nutrient_array = [df['Protein'][row], df['Fats'][row], df['Energy'][row], df['Carbs'][row], df['Fibres'][row]]
    print(nutrient_array)
    return nutrient_array

nutrient_array = create_input(1005)

[2, 5, 72, 6, 3]


In [81]:
# taking only nutrients as input
nearest_recipes = model.kneighbors(np.array(nutrient_array).reshape(1, -1), 10, return_distance=False)

In [82]:
# nearest_recipes = neigh.kneighbors(input_arr.reshape(1, -1), 10, return_distance=False)
indices = nearest_recipes[0]
df2 = df.iloc[indices, :]
indices

array([1005, 2703, 2858, 2683,  885, 1425, 1492, 2542, 1279,    4])

In [83]:
df2

,Name,Protein,Fats,Energy,Carbs,Fibres,image_url,ingridients,mealtype,url,...,nordic,south american,south east asian,world,Balanced,High-Fiber,High-Protein,Low-Carb,Low-Fat,Low-Sodium
1005,Za'atar recipes,2,5,72,6,3,https://edamam-product-images.s3.amazonaws.com...,"['4 tablespoons fresh thyme leaves, stripped f...",snack,http://www.101cookbooks.com/archives/zaatar-re...,...,0,0,0,0,0,0,0,0,0,1
2703,Sichuan-Style Smashed Cucumber Salad Recipe,2,2,74,12,1,https://edamam-product-images.s3.amazonaws.com...,"['1 whole English cucumber, washed, peeled in ...",lunch/dinner,https://www.seriouseats.com/sichuan-style-smas...,...,0,0,0,0,1,0,0,0,0,0
2858,The Old Hickory Cocktail Recipe,0,0,75,2,0,https://edamam-product-images.s3.amazonaws.com...,"['1 1/2 ounces dry vermouth', '1 1/2 ounces sw...",lunch/dinner,https://www.seriouseats.com/classic-cocktail-o...,...,0,0,0,1,0,0,0,0,1,1
2683,Time for a Drink: Fallen Leaves Recipe,0,0,73,0,0,https://edamam-product-images.s3.amazonaws.com...,"['3/4 ounce Calvados or other apple brandy', '...",lunch/dinner,http://www.seriouseats.com/recipes/2007/10/coc...,...,0,0,0,1,0,0,0,0,1,1
885,Pickled Hakurei Turnips Recipe,1,0,76,13,2,https://edamam-product-images.s3.amazonaws.com...,"['1 bunch hakurei turnips (approximately six, ...",snack,http://www.seriouseats.com/recipes/2012/05/pic...,...,0,0,0,0,0,0,0,0,1,0
1425,The Most Basic Chicken Stock,2,5,65,0,0,https://edamam-product-images.s3.amazonaws.com...,"['1 chicken carcass or more', '1 gallon water'...",snack,https://food52.com/recipes/3447-the-most-basic...,...,0,0,0,0,0,0,0,1,0,1
1492,Shucked Oysters with Three Sauces,1,0,67,13,2,https://edamam-product-images.s3.amazonaws.com...,"['3 ounces fresh horseradish, peeled', '1 teas...",snack,https://www.marthastewart.com/1544318/shucked-...,...,0,0,0,0,0,0,0,0,1,1
2542,The Crispy Egg,5,4,65,0,0,https://edamam-product-images.s3.amazonaws.com...,['1 glug olive oil or cooking fat of your choi...,lunch/dinner,http://smittenkitchen.com/blog/2014/10/the-cri...,...,0,0,0,0,0,0,0,1,0,1
1279,Raw Garlic Sauce,3,0,71,15,1,https://edamam-product-images.s3.amazonaws.com...,"['16 cloves minced fresh garlic', '1⁄4 tsp. le...",snack,https://www.saveur.com/raw-garlic-sauce-recipe/,...,0,0,0,0,0,0,0,0,1,1
4,Sweet Rice Flour Porridge from 'The Kimchi Coo...,1,0,72,15,0,https://edamam-product-images.s3.amazonaws.com...,"['1 cup water, divided', '2 tablespoons sweet ...","breakfast,lunch/dinner",http://www.seriouseats.com/recipes/2013/01/swe...,...,0,0,0,0,0,0,0,0,1,1


Apply nearest neighbours - use mealtype and cuisinetype

In [84]:
# use nutrients, mealtype, cuisinetype for finding nearest neighbours
neigh.fit(df.iloc[:, list(range(1, 6)) + list(range(12, 38))].to_numpy())

NearestNeighbors()

saving the 2nd model

In [85]:
with open('recipemodel2', 'wb') as file:
    pickle.dump(neigh, file)

In [86]:
model2 = pickle.load(open('recipemodel2','rb'))

In [87]:
def create_input(row):
    nutrient_array = [df['Protein'][row], df['Fats'][row], df['Energy'][row], df['Carbs'][row], df['Fibres'][row]]
    print(nutrient_array)

    meal_array = []
    for column_name in df.columns[12:17]:
        meal_array.append(column_name)
    specific_row = df['mealtype'][row]
    result_meal_array = [1 if category in specific_row else 0 for category in meal_array]
    print(result_meal_array)

    cuisine_array=[]
    for column_name in df.columns[17:38]:
        cuisine_array.append(column_name)
    print(cuisine_array)
    specific_row = df['cuisinetype'][row]
    result_cuisine_array = [1 if category in specific_row else 0 for category in cuisine_array]
    print(result_cuisine_array)

    input_arr = np.append(np.append(np.array(nutrient_array), np.array(result_meal_array)), np.array(result_cuisine_array))
    input_arr = np.array(input_arr)
    print(input_arr)
    return input_arr

input_arr = create_input(1005)

[2, 5, 72, 6, 3]
[0, 0, 0, 1, 0]
['american', 'asian', 'british', 'caribbean', 'central europe', 'chinese', 'eastern europe', 'french', 'greek', 'indian', 'italian', 'japanese', 'korean', 'kosher', 'mediterranean', 'mexican', 'middle eastern', 'nordic', 'south american', 'south east asian', 'world']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[ 2  5 72  6  3  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  0  0  0  0  0]


In [88]:
# taking mealtype, cuisinetype and nutrients as input
nearest_recipes2 = model2.kneighbors(input_arr.reshape(1, -1), 10, return_distance=False)

In [89]:
# nearest_recipes = neigh.kneighbors(input_arr.reshape(1, -1), 10, return_distance=False)
indices2 = nearest_recipes2[0]
df3 = df.iloc[indices2, :]
indices2

array([1005, 2703, 2858, 2683,  885, 1425, 1492, 2542, 1279,    4])

In [90]:
df3

,Name,Protein,Fats,Energy,Carbs,Fibres,image_url,ingridients,mealtype,url,...,nordic,south american,south east asian,world,Balanced,High-Fiber,High-Protein,Low-Carb,Low-Fat,Low-Sodium
1005,Za'atar recipes,2,5,72,6,3,https://edamam-product-images.s3.amazonaws.com...,"['4 tablespoons fresh thyme leaves, stripped f...",snack,http://www.101cookbooks.com/archives/zaatar-re...,...,0,0,0,0,0,0,0,0,0,1
2703,Sichuan-Style Smashed Cucumber Salad Recipe,2,2,74,12,1,https://edamam-product-images.s3.amazonaws.com...,"['1 whole English cucumber, washed, peeled in ...",lunch/dinner,https://www.seriouseats.com/sichuan-style-smas...,...,0,0,0,0,1,0,0,0,0,0
2858,The Old Hickory Cocktail Recipe,0,0,75,2,0,https://edamam-product-images.s3.amazonaws.com...,"['1 1/2 ounces dry vermouth', '1 1/2 ounces sw...",lunch/dinner,https://www.seriouseats.com/classic-cocktail-o...,...,0,0,0,1,0,0,0,0,1,1
2683,Time for a Drink: Fallen Leaves Recipe,0,0,73,0,0,https://edamam-product-images.s3.amazonaws.com...,"['3/4 ounce Calvados or other apple brandy', '...",lunch/dinner,http://www.seriouseats.com/recipes/2007/10/coc...,...,0,0,0,1,0,0,0,0,1,1
885,Pickled Hakurei Turnips Recipe,1,0,76,13,2,https://edamam-product-images.s3.amazonaws.com...,"['1 bunch hakurei turnips (approximately six, ...",snack,http://www.seriouseats.com/recipes/2012/05/pic...,...,0,0,0,0,0,0,0,0,1,0
1425,The Most Basic Chicken Stock,2,5,65,0,0,https://edamam-product-images.s3.amazonaws.com...,"['1 chicken carcass or more', '1 gallon water'...",snack,https://food52.com/recipes/3447-the-most-basic...,...,0,0,0,0,0,0,0,1,0,1
1492,Shucked Oysters with Three Sauces,1,0,67,13,2,https://edamam-product-images.s3.amazonaws.com...,"['3 ounces fresh horseradish, peeled', '1 teas...",snack,https://www.marthastewart.com/1544318/shucked-...,...,0,0,0,0,0,0,0,0,1,1
2542,The Crispy Egg,5,4,65,0,0,https://edamam-product-images.s3.amazonaws.com...,['1 glug olive oil or cooking fat of your choi...,lunch/dinner,http://smittenkitchen.com/blog/2014/10/the-cri...,...,0,0,0,0,0,0,0,1,0,1
1279,Raw Garlic Sauce,3,0,71,15,1,https://edamam-product-images.s3.amazonaws.com...,"['16 cloves minced fresh garlic', '1⁄4 tsp. le...",snack,https://www.saveur.com/raw-garlic-sauce-recipe/,...,0,0,0,0,0,0,0,0,1,1
4,Sweet Rice Flour Porridge from 'The Kimchi Coo...,1,0,72,15,0,https://edamam-product-images.s3.amazonaws.com...,"['1 cup water, divided', '2 tablespoons sweet ...","breakfast,lunch/dinner",http://www.seriouseats.com/recipes/2013/01/swe...,...,0,0,0,0,0,0,0,0,1,1


In [2]:
#  The second model produces worse or the same results and has a larger dataset so more overhead, to get better and fast results model 1 was used for final predictions in backend
#  which only contains the 5 nutrient columns